# Sear for Bad spectra 

- look for which parameter allow to identify a bad spectrum

 work with Weakly_2023_11
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/03/05
- last update : 2023/04/05

- Spectractor https://github.com/lsst/Spectractor/
- Spectractor Version 2.4
- branch lsst-dev
- available in version w_2023_11 by default


In [ ]:
! eups list -s | grep LOCAL

In [ ]:
! eups list -s lsst_distrib

In [ ]:
import h5py

In [ ]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as tck
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
import pickle

In [ ]:
from astropy.time import Time
import astropy.units as u

## Config

In [ ]:
FLAG_MOUNT_ERRORS= True

### Butler

In [ ]:
#repo =  "/sdf/group/rubin/repo/main"
#repo = "/sdf/group/rubin/repo/oga/"
repo = "/repo/embargo/"
butler = dafButler.Butler(repo)
registry = butler.registry

### Date & Filter & Disperser & collection

- make your choice

In [ ]:
DateToRuncollection = {

 # september 2022
 20220912 :  "u/dagoret/BPS_manyspectro_v60", # Missing collection
 20220913 :  "u/dagoret/BPS_manyspectro_v61", # Missing collection
 20220914 :  "u/dagoret/BPS_manyspectro_v62", # Missing collection  
    
 20220927 :  "u/dagoret/BPS_manyspectro_v63", # Mount errors available, mitigé
 20220928 :  "u/dagoret/BPS_manyspectro_v64", # Mount errors available, very nice spectra
 20220929 :  "u/dagoret/BPS_manyspectro_v65", # Mount errors available, very nice spectra
    
    
 # october 2022
 20221012 :  "u/dagoret/BPS_manyspectro_v56", # mount errors , nice
 20221025 :  "u/dagoret/BPS_manyspectro_v57", # mount errors, very bad spectra
 20221026 :  "u/dagoret/BPS_manyspectro_v58", # mount errors,  nice spectra
 20221027 :  "u/dagoret/BPS_manyspectro_v59", # mount errors,  nice spectra
    
 # november 2022
 20221109 :  "u/dagoret/BPS_manyspectro_v53", # nice
 20221110 :  "u/dagoret/BPS_manyspectro_v54", # mount errors available, many bad rec spectra
 20221123 :  "u/dagoret/BPS_manyspectro_v55", # mount errors , nice   
    
 # december 2022   
 20221207 :  "u/dagoret/BPS_manyspectro_v48",
 20221208 :  "u/dagoret/BPS_manyspectro_v49",
 20221209 :  "u/dagoret/BPS_manyspectro_v50", 
 20221210 :  "u/dagoret/BPS_manyspectro_v51", 
 20221212 :  "u/dagoret/BPS_manyspectro_v52",    
    
 #January 2023 
 20230117 :  "u/dagoret/BPS_manyspectro_v79", # no mount errors
 20230118 :  "u/dagoret/BPS_manyspectro_v80", # no mount error
 20230119 :  "u/dagoret/BPS_manyspectro_v77", # no mount errors
 20230131 :  "u/dagoret/BPS_manyspectro_v78", # missing for moment

 #february 2023
 20230214 :  "u/dagoret/BPS_manyspectro_v42",
 20230215 :  "u/dagoret/BPS_manyspectro_v43", # mount errors
 20230216 :  "u/dagoret/BPS_manyspectro_v44", # mount errors 
 20230228 :  "u/dagoret/BPS_manyspectro_v70",
    
 #march 2023  
 20230301 :  "u/dagoret/BPS_manyspectro_v71", 
 20230302 :  "u/dagoret/BPS_manyspectro_v72", 
 20230315 :  "u/dagoret/BPS_manyspectro_v73", 
 20230316 :  "u/dagoret/BPS_manyspectro_v74",    
    
}

In [ ]:
DATE = 20230316
my_collection = DateToRuncollection[DATE]

filterdispersername = "empty~holo4_003"

RUNSELECTION = 20230330
#RUNSELECTION = 20230403

# List of Dataset types avalaible in my collection

## List of dataset types

In [ ]:
list(registry.queryDatasetTypes("*spec*"))

## list of datasets

In [ ]:
list(registry.queryDatasets("*spec*", collections=[my_collection]))[:30]

### Spectractor

In [ ]:
configmode = "PSF2DFFM_REBIN2"
#configmode = "PSF2DFFM_REBIN1"
#ext="_b"
specver="specV2.4"

https://www.pythonforthelab.com/blog/how-to-use-hdf5-files-in-python/

In [ ]:
output_file_h5 = f"allspectra_{DATE}-{filterdispersername}-{configmode}-{specver}-oga.hdf5"
input_summary_file = f"fullsummaryspectra_{DATE}-{filterdispersername}-{configmode}-{specver}-oga.csv"

### The collection

In [ ]:
datasetRefs = registry.queryDatasets(datasetType='spectractorSpectrum', collections=my_collection, where= "instrument='LATISS'")

In [ ]:
# following line not possible in OGA
#butler = butlerUtils.makeDefaultLatissButler(extraCollections=[my_collection])

## Read Mout Error summary file

In [ ]:
if FLAG_MOUNT_ERRORS:
    df_mounterrors = pd.read_csv("mountErrors.csv")
    df=df_mounterrors[df_mounterrors["date_obs"] == DATE ]
    if len(df) == 0:
        FLAG_MOUNT_ERRORS = False
        

In [ ]:
df_mounterrors

In [ ]:
df

In [ ]:
FLAG_MOUNT_ERRORS 

## Read full summary files

In [ ]:
df_summary=pd.read_csv(input_summary_file,index_col=0)

In [ ]:
df_summary

## Loop on exposures

In [ ]:
#outfile.close()

In [ ]:
all_dataId = []     # full data id
all_spec = []       # spectra
all_exposures = []  # exposure number
all_num = []        # sequence numbers


for i, ref in enumerate(datasetRefs):

    print(f"============= ({i}) ============datasetType = spectraction ============================================")
    print("fullId..................:",ref.dataId.full)
    print("visit...................:",ref.dataId["visit"])
    print("band....................:",ref.dataId["band"])
    print("physical filter.........:",ref.dataId["physical_filter"])
    print("run.....................:",ref.run)
    
    #through old runs
    date_of_run = int(ref.run.split("/")[-1].split("T")[0])
    if date_of_run< RUNSELECTION :
        continue
 
    
    
    the_exposure = ref.dataId["visit"]
    the_day_obs = ref.dataId["visit"]//100_000
    the_seq_num = ref.dataId["visit"]- the_day_obs*100_000    
    the_dataId = {'day_obs': the_day_obs,'seq_num':the_seq_num,'detector':0}
    print(the_dataId)
    
    the_exposure_key = str(the_exposure)
    
    
    
    # retrieve the spectrum from Butler
    #spec       = butler.get('spectraction',the_dataId)
    spec = butler.get('spectractorSpectrum', visit=the_exposure, detector=0, collections=my_collection, instrument='LATISS')
    
    # save in collection lists
    all_dataId.append(the_dataId) 
    all_exposures.append(the_exposure)
    all_spec.append(spec)
    all_num.append(the_seq_num)
    

In [ ]:
index = 0

spec = all_spec[index]

In [ ]:
def DumpSpectrum(spec):
    print("--------------------------------------------")
    print("spec.target.label\t",spec.target.label)
    print("spec.filter_label\t",spec.filter_label)
    print("spec.airmass\t",spec.airmass)
    print("spec.date_obs\t",spec.date_obs)
    print("spec.temperature\t",spec.temperature)
    print("spec.pressure\t",spec.pressure)
    print("spec.humidity\t",spec.humidity)    
    #print("spec.ra\t",spec.ra)
    print("spec.dec\t",spec.dec)
    print("spec.hour_angle\t",spec.hour_angle)
    print("spec.parallactic_angle\t",spec.parallactic_angle)
    print("spec.camera_angle\t", spec.camera_angle)
    print("--------------------------------------------")
    print("spec.order\t",spec.order)
    print("spec.lambdas\t",spec.lambdas)
    print("spec.data\t",spec.data)
    print("spec.err\t",spec.err)
    print("spec.covmatrix\t",spec.cov_matrix)
    print("--------------------------------------------")
    print("spec.data_next_order\t",spec.data_next_order)
    print("spec.err_next_order\t",spec.err_next_order)

In [ ]:
DumpSpectrum(spec)

## Plot

In [ ]:
chi2_fit_max = 25
chi2_fit_min = 0

d2ccd_max = 188
d2ccd_min = 186.6

pixshift_max = 0.3
pixshift_min = -0.3

In [ ]:
all_histnames = ['spec_header_chi2_fit','spec_header_d2ccd','spec_header_pixshift']
all_histminval = [chi2_fit_min, d2ccd_min, pixshift_min ]
all_histmaxval = [chi2_fit_max, d2ccd_max, pixshift_max ]

In [ ]:
%matplotlib inline


infos = []
all_lambdas=[]
all_data=[]
all_data_err=[]


idx=0
for spec in all_spec:
    
    
    #s=spec.spectrum
    s=spec
    
    the_exposure = all_exposures[idx]
    
    label = str(idx) +"):" + str(all_exposures[idx])
    
    try:
        fig=plt.figure(figsize=(16,8))
    
        ax1 = fig.add_subplot(3, 2, 1)
        s.plot_spectrum(ax=ax1,force_lines=True,label=label)
    
        ax2 = fig.add_subplot(3, 2, 2)
        s.plot_spectrogram(ax=ax2,scale="log")
    
        all_lambdas.append(s.lambdas)
        all_data.append(s.data)
        all_data_err.append(s.err)
        
        
        df_theexpo = df_summary[df_summary['exposure'] == the_exposure]
        
        ax3 = fig.add_subplot(3, 2, 3)
        i=0
        name = all_histnames[i]
        nbins=50
        values = df_summary[name].values
        values0 = df_theexpo[name].values
        ax3.hist(values,bins=nbins,facecolor="blue")
        ax3.hist(values0,bins=nbins,facecolor="red")
        ax3.set_title(name)
        ax3.grid()
        ax3.xaxis.set_minor_locator(tck.AutoMinorLocator())
        ax3.axvline(all_histminval[i],color="g")
        ax3.axvline(all_histmaxval[i],color="g")
        ax3.axvline(values0[0],color="r")
        
        ax4 = fig.add_subplot(3, 2, 4)
        i=2
        name = all_histnames[i]
        nbins=50
        values = df_summary[name].values
        values0 = df_theexpo[name].values
        ax4.hist(values,bins=nbins,facecolor="blue")
        ax4.hist(values0,bins=nbins,facecolor="red")
        ax4.set_title(name)
        ax4.grid()
        ax4.xaxis.set_minor_locator(tck.AutoMinorLocator())
        ax4.axvline(all_histminval[i],color="g")
        ax4.axvline(all_histmaxval[i],color="g")
        ax4.axvline(values0[0],color="r")
        
        
        ax5 = fig.add_subplot(3, 2, 5)
        i=1
        name = all_histnames[i]
        nbins=50
        values = df_summary[name].values
        values0 = df_theexpo[name].values
        ax5.hist(values,bins=nbins,facecolor="blue")
        ax5.hist(values0,bins=nbins,facecolor="red")
        ax5.set_title(name)
        ax5.grid()
        ax5.xaxis.set_minor_locator(tck.AutoMinorLocator())
        ax5.axvline(all_histminval[i],color="g")
        ax5.axvline(all_histmaxval[i],color="g")
        ax5.axvline(values0[0],color="r")
        
        
        ax6 = fig.add_subplot(3,2,6)
        xpix=df_summary["spec_header_targetx"].values
        ypix=df_summary["spec_header_targety"].values
        xpix0=df_theexpo["spec_header_targetx"].values
        ypix0=df_theexpo["spec_header_targety"].values
        ax6.scatter(xpix,ypix,color="blue",marker='+',s=20)
        ax6.scatter(xpix0,ypix0,color="red",marker='o',s=30)
        ax6.set_xlabel("targetx")
        ax6.set_ylabel("targety")
        ax6.xaxis.set_minor_locator(tck.AutoMinorLocator())
        ax6.yaxis.set_minor_locator(tck.AutoMinorLocator())
        ax6.grid()
        
        plt.tight_layout()
        
    
        
            
    except Exception as inst:
        print(" >>> Exception catched for "+ label )
        print(type(inst))    # the exception instance
        print(inst.args)     # arguments stored in .args

    
    idx+=1

In [ ]:
df_summary[['exposure','spec_header_chi2_fit','spec_header_d2ccd','spec_header_pixshift']].sort_values(by="exposure")